<a href="https://colab.research.google.com/github/TheDarkNight21/blues-clues---HVAC-note-extractor/blob/main/train_detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# step 1: load datasets

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

dataset_upscaled_dir = '/content/gdrive/My Drive/blues clues/datasets/blueprint_notes_upscaled'
dataset_rescaled_dir = '/content/gdrive/My Drive/blues clues/datasets/blueprint_notes_rescaled'
dataset_notes_dir = '/content/gdrive/My Drive/blues clues/datasets/blueprint_notes'

Mounted at /content/gdrive


In [2]:
import os

drive_path = dataset_upscaled_dir

if os.path.isdir(drive_path):
    print(f"'{drive_path}' is mounted and accessible.")
    print("Files in My Drive:", os.listdir(drive_path)[:10]) # Print first 10 files/folders
else:
    print(f"'{drive_path}' is not accessible. Drive may not be mounted correctly.")

'/content/gdrive/My Drive/blues clues/datasets/blueprint_notes_upscaled' is mounted and accessible.
Files in My Drive: ['annotations']


# step 2: load detectron2 model and necessary functions

In [3]:
# Install detectron2 dependencies
!pip install torch torchvision

# Install detectron2 from source (recommended for Colab)
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Restart runtime after installation (Colab menu: Runtime > Restart runtime)

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-gk9rawrz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-gk9rawrz
  Resolved https://github.com/facebookresearch/detectron2.git to commit fd27788985af0f4ca800bca563acdb700bb890e2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 18.5 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=7085024 sha25

In [5]:
# --- 1. Register Datasets ---

from detectron2.data.datasets import register_coco_instances

register_coco_instances(
    "blueprint_train",
    {},
    f"{dataset_rescaled_dir}/annotations/instances_train.json",
    f"{dataset_rescaled_dir}/train"
)

register_coco_instances(
    "blueprint_val",
    {},
    f"{dataset_rescaled_dir}/annotations/instances_val.json",
    f"{dataset_rescaled_dir}/val"
)

In [9]:
# --- 2. Setup Configuration ---

from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

cfg.MODEL.DEVICE = "cuda"
print(f"Using device: {cfg.MODEL.DEVICE}")

cfg.DATASETS.TRAIN = ("blueprint_train",)
cfg.DATASETS.TEST = ("blueprint_val",)
cfg.DATALOADER.NUM_WORKERS = 2 # good for t4 gpu

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

cfg.SOLVER.LOG_PERIOD = 50
cfg.TEST.EVAL_PERIOD = 500

cfg.OUTPUT_DIR = "/content/drive/MyDrive/output_blueprints"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

Using device: cuda


In [11]:
# --- 3. Train the Model ---

from detectron2.engine import DefaultTrainer

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

print("\nTraining finished.")
print(f"To see training graphs, run the following command in your terminal:")
print(f"tensorboard --logdir {cfg.OUTPUT_DIR}")


[01/19 16:21:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:00, 240MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[01/19 16:21:15 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W0119 16:21:19.595000 424 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[01/19 16:21:30 d2.utils.events]:  eta: 0:08:39  iter: 19  total_loss: 1.002  loss_cls: 0.8066  loss_box_reg: 0.01142  loss_rpn_cls: 0.1577  loss_rpn_loc: 0.01038    time: 0.5496  last_time: 0.5086  data_time: 0.2163  last_data_time: 0.1132   lr: 4.9953e-06  max_mem: 2424M
[01/19 16:21:46 d2.utils.events]:  eta: 0:08:13  iter: 39  total_loss: 0.9842  loss_cls: 0.7066  loss_box_reg: 0.06188  loss_rpn_cls: 0.191  loss_rpn_loc: 0.009734    time: 0.5254  last_time: 0.4584  data_time: 0.1022  last_data_time: 0.0072   lr: 9.9902e-06  max_mem: 2424M
[01/19 16:21:54 d2.utils.events]:  eta: 0:07:06  iter: 59  total_loss: 0.775  loss_cls: 0.5453  loss_box_reg: 0.07122  loss_rpn_cls: 0.1217  loss_rpn_loc: 0.009123    time: 0.4865  last_time: 0.4929  data_time: 0.0075  last_data_time: 0.0121   lr: 1.4985e-05  max_mem: 2424M
[01/19 16:22:03 d2.utils.events]:  eta: 0:06:46  iter: 79  total_loss: 0.6286  loss_cls: 0.4091  loss_box_reg: 0.1067  loss_rpn_cls: 0.09351  loss_rpn_loc: 0.009418    time: 0.

In [12]:
print(f"Model saved to: {cfg.OUTPUT_DIR}")
print(f"Final model: {cfg.OUTPUT_DIR}/model_final.pth")

Model saved to: /content/drive/MyDrive/output_blueprints
Final model: /content/drive/MyDrive/output_blueprints/model_final.pth
